Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [53]:
from itertools import combinations
from collections import namedtuple
import numpy as np
import random
import os
import math
from typing import List, Tuple
from tqdm import tqdm

## Simple Test Problem

In [54]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')
NUM_CITIES = test_problem.shape[0]

In [55]:
def analyze_matrix(problem):
    has_negative = np.any(problem < 0)
    has_zero_diag = np.allclose(np.diag(problem), 0.0)
    is_symmetric = np.allclose(problem, problem.T)

    print("Common tests -> negatives:", has_negative,
    "\n zero diagonal:", has_zero_diag,
    "\n symmetric:", is_symmetric)
    return{
        "is_symmetric": is_symmetric,
        "has_negative": has_negative,
        "has_zero_diag": has_zero_diag
    }

UTILITY FUNCTIONS
- fitness function
- nearest_neighbor_tour function uses greedy heuristics to find initial tours.
- two_optimum function applies 2-optimum local improvement

In [56]:
def fitness(route, matrix):
    n_cities = len(route)
    cost = 0.0
    for i in range(n_cities):
        cost += matrix[route[i], route[(i + 1) % n_cities]]
    return cost


def nearest_neighbor_tour(matrix, start=None):
    n_cities = len(matrix)
    unvisited = set(range(n_cities))

    if start is None:
        start = random.randrange(n_cities)
    route = [start]
    unvisited.remove(start)
    current = start
    while unvisited:
        next_city = min(unvisited, key=lambda j: matrix[current, j])
        route.append(next_city)
        unvisited.remove(next_city)
        current = next_city
    return route

def two_opt(route, matrix):
    best = route
    best_cost = fitness(best, matrix)
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1: 
                    continue
                new_route = best[:i] + best[i:j][::-1] + best[j:]
                new_cost = fitness(new_route, matrix)
                if new_cost < best_cost:
                    best, best_cost = new_route, new_cost
                    improved = True
        route = best
    return best, best_cost


GENETIC OPERATORS
- segmented_crossover function exchanges random segments between parents to form a child
- swap_mutation function swaps two random cities
- inversion_mutation function reverses a random segment
- shift_mutation function shifts a random segment by one position
- fix_route function ensure no duplicates or missing cities 

In [57]:
def segmented_crossover(p1, p2):
    n = len(p1)
    cut1, cut2 = sorted(random.sample(range(n), 2))
    segment = p1[cut1:cut2]
    remaining = [x for x in p2 if x not in segment]
    child = remaining[:cut1] + segment + remaining[cut1:]
    return child

def swap_mutation(tour):
    a, b = random.sample(range(len(tour)), 2)
    tour[a], tour[b] = tour[b], tour[a]
    return tour

def inversion_mutation(tour):
    a, b = sorted(random.sample(range(len(tour)), 2))
    tour[a:b] = reversed(tour[a:b])
    return tour

def shift_mutation(tour):
    a, b = sorted(random.sample(range(len(tour)), 2))
    seg = tour[a:b]
    del tour[a:b]
    insert_pos = random.randint(0, len(tour))
    for i, x in enumerate(seg):
        tour.insert((insert_pos + i) % len(tour), x)
    return tour

def fix_route(route):
    n = len(route)
    seen = set()
    fixed = []
    missing = [x for x in range(n) if x not in route]
    miss_i = 0
    for x in route:
        if x in seen:
            fixed.append(missing[miss_i])
            miss_i+1
        else: 
            fixed.append(x)
            seen.add(x)
    return fixed

EVOLUTIONARY ALGORITHM
Evolutionary loop for the TSP solver using selection, crossover, mutation and elitism to find better routes.
It includes adaptive tuning, adjusting mutation rate dynamically and progressively and ocasionally applies a 2-optimum local refinement.

In [58]:
def evolutionary_tsp(
    matrix,
    info,
    population_size=60,
    generations=300,
    elite_frac=0.1,
    base_mutation_rate=0.2,
    local_refine_interval=25,
    verbose=True
):
    n = len(matrix)
    num_elite = max(1, int(elite_frac * population_size))

    has_negative = info["has_negative"]

    mutation_rate = base_mutation_rate * (2.0 if has_negative else 1.0) #double mutation rate for negative weigths
    refine_interval = local_refine_interval if not has_negative else 9999 #to skip 2-optimum local search
    
    # Initialize population as half random and half greedy
    population = []
    for _ in range(population_size // 2):
        population.append(random.sample(range(n), n))
    for _ in range(population_size - len(population)):
        population.append(nearest_neighbor_tour(matrix))
    
    fit = np.array([fitness(t, matrix) for t in population])
    best_idx = np.argmin(fit)
    best_route = population[best_idx][:]
    best_score = fit[best_idx]
    best_history = [best_score]

    
    for gen in tqdm(range(generations), desc="Evolving"):
        # tournament of 3
        new_pop = []
        for _ in range(population_size - num_elite):
            contenders = random.sample(range(population_size), 3)
            p1 = min(contenders, key=lambda i: fit[i])
            contenders = random.sample(range(population_size), 3)
            p2 = min(contenders, key=lambda i: fit[i])
            
            child = segmented_crossover(population[p1], population[p2])
            
            if random.random() < mutation_rate:
                op = random.choices([swap_mutation, inversion_mutation, shift_mutation],
                    weights=[0.4, 0.4, 0.2])[0]
                child = op(child) 
            new_pop.append(child)
        
        # keep best routes
        elites_idx = np.argsort(fit)[:num_elite]
        new_pop.extend([population[i][:] for i in elites_idx])
        
        population = new_pop
        fit = np.array([fitness(t, matrix) for t in population])
        
        current_best_idx = np.argmin(fit)
        current_best_score = fit[current_best_idx]
        
        if current_best_score < best_score:
            best_score = current_best_score
            best_route = population[current_best_idx][:]
        
        if gen % 50 == 0 and gen > 0:
            recent = best_history[-50:]
            if len(recent) > 5 and np.std(recent[-5:]) < 1e-5:
                mutation_rate = min(0.9, mutation_rate * 1.5)
            else:
                mutation_rate = max(0.05, mutation_rate * 0.9)
        
        if gen % refine_interval == 0:
            best_route, best_score = two_opt(best_route, matrix)
        
        best_history.append(best_score)
        
        if verbose and gen % 50 == 0:
            print(f"Gen {gen:4d} | Best cost: {best_score:.3f} | MutRate: {mutation_rate:.2f}")
    
    return best_route, best_score, best_history

TEST RUNNER
- run_npy_tests function loads all .npy files and runs the TSP solver

In [59]:

def run_npy_tests(folder="lab2", generations=300):
    files = [f for f in os.listdir(folder) if f.endswith(".npy")]
    
    results = []
    
    for fname in sorted(files):
        path = os.path.join(folder, fname)
        print(f"\n Running test: {fname}")
        
        matrix = np.load(path)

        info = analyze_matrix(matrix)
        print(f"Matrix Analysis: {info}")

        route, cost, _ = evolutionary_tsp(matrix, info, generations=generations, verbose=False)
        
        print(f"Finished {fname} \n| Cities: {len(matrix)} \n| Best cost: {cost:.3f}")
        results.append((fname, len(matrix), cost))
    
    print("\n Summary:")
    for fname, n, cost in results:
        print(f"{fname:<20} | n={n:<5d} | best={cost:.3f}")
    
    return results



if __name__ == "__main__":
    np.random.seed(42)
    random.seed(42)
    
    results = run_npy_tests(folder="lab2", generations=300)


 Running test: problem_g_10.npy
Common tests -> negatives: False 
 zero diagonal: True 
 symmetric: True
Matrix Analysis: {'is_symmetric': True, 'has_negative': np.False_, 'has_zero_diag': True}


Evolving: 100%|██████████| 300/300 [00:00<00:00, 1347.51it/s]


Finished problem_g_10.npy 
| Cities: 10 
| Best cost: 1497.664

 Running test: problem_g_100.npy
Common tests -> negatives: False 
 zero diagonal: True 
 symmetric: True
Matrix Analysis: {'is_symmetric': True, 'has_negative': np.False_, 'has_zero_diag': True}


Evolving: 100%|██████████| 300/300 [00:01<00:00, 159.06it/s]


Finished problem_g_100.npy 
| Cities: 100 
| Best cost: 4171.107

 Running test: problem_g_1000.npy
Common tests -> negatives: False 
 zero diagonal: True 
 symmetric: True
Matrix Analysis: {'is_symmetric': True, 'has_negative': np.False_, 'has_zero_diag': True}


...

Finished problem_r2_50.npy 
| Cities: 50 
| Best cost: -2232.470

 Running test: problem_r2_500.npy
Common tests -> negatives: True 
 zero diagonal: False 
 symmetric: False
Matrix Analysis: {'is_symmetric': False, 'has_negative': np.True_, 'has_zero_diag': False}


Evolving: 100%|██████████| 300/300 [00:22<00:00, 13.21it/s]


Finished problem_r2_500.npy 
| Cities: 500 
| Best cost: -24518.609

 Running test: test_problem.npy
Common tests -> negatives: False 
 zero diagonal: True 
 symmetric: True
Matrix Analysis: {'is_symmetric': True, 'has_negative': np.False_, 'has_zero_diag': True}


Evolving: 100%|██████████| 300/300 [00:00<00:00, 940.94it/s]

Finished test_problem.npy 
| Cities: 20 
| Best cost: 2823.790

 Summary:
problem_g_10.npy     | n=10    | best=1497.664
problem_g_100.npy    | n=100   | best=4171.107
problem_g_1000.npy   | n=1000  | best=12498.301
problem_g_20.npy     | n=20    | best=1755.515
problem_g_200.npy    | n=200   | best=5893.105
problem_g_50.npy     | n=50    | best=2678.977
problem_g_500.npy    | n=500   | best=8828.399
problem_r1_10.npy    | n=10    | best=184.273
problem_r1_100.npy   | n=100   | best=738.806
problem_r1_1000.npy  | n=1000  | best=2534.690
problem_r1_20.npy    | n=20    | best=342.422
problem_r1_200.npy   | n=200   | best=1108.260
problem_r1_50.npy    | n=50    | best=607.106
problem_r1_500.npy   | n=500   | best=1739.918
problem_r2_10.npy    | n=10    | best=-411.702
problem_r2_100.npy   | n=100   | best=-4670.683
problem_r2_1000.npy  | n=1000  | best=-49385.994
problem_r2_20.npy    | n=20    | best=-815.898
problem_r2_200.npy   | n=200   | best=-9589.934
problem_r2_50.npy    | n=50    |